In [2]:
# https://www.les-sports.info/2021-tokyo-jeux-olympiques-d-ete-s16-c0-b0-j0-u323.html

In [3]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from time import sleep

### Récupération du tableau contenant les données sur une page du site

In [4]:
# Get the data from the website
url = 'https://www.les-sports.info/2021-tokyo-jeux-olympiques-d-ete-s16-c0-b0-j0-u323.html'
page = requests.get(url)
soup = bs(page.content, 'html.parser')

# # Get the table from the website
# table = soup.find('table', {'class': 'table-style-2'}, )
# rows = table.find_all('tr')

In [5]:
tables = soup.find_all('table', {'class': 'table-style-2'})
len(tables)

2

In [6]:
table = tables[1]
table

<table class="table-style-2">
<tr><th class="td-center">Place</th><th class="td-left">Pays</th><th class="td-center"><img alt="Or" class="flag3" src="images/gold_medal.png"/>Or</th><th class="td-center"><img alt="Argent" class="flag3" src="images/silver_medal.png"/>Argent</th><th class="td-center"><img alt="Bronze" class="flag3" src="images/bronze_medal.png"/>Bronze</th><th class="td-center">Total</th></tr>
<tr onclick="maj_medailles_multi_sport('unitedstates', '15', '0', '323', 'E', 'fr')" style="cursor: pointer;"><td class="tdcol-7 td-center">1</td><td class="td-left tdcol-40"><img alt="Etats Unis" class="flag" src="images/flags/15.png"/>Etats Unis</td><td class="tdcol-10 td-center">38</td><td class="tdcol-10 td-center">41</td><td class="tdcol-10 td-center">34</td><td class="tdcol-10 td-center">113</td></tr>
<tr class="contenu_medailles"><td colspan="6" id="unitedstates"></td></tr>
<tr onclick="maj_medailles_multi_sport('china', '65', '0', '323', 'E', 'fr')" style="cursor: pointer;">

##### Extraction des colonnes de la table

In [7]:
col = [td.get_text().strip() for td in table.select("th")]
col

['Place', 'Pays', 'Or', 'Argent', 'Bronze', 'Total']

##### Elements de la table

In [8]:
ele = [td.get_text() for td in table.select("td")]
print(ele)


['1', 'Etats Unis', '38', '41', '34', '113', '', '2', 'Chine', '36', '32', '18', '86', '', '3', 'Japon', '27', '12', '16', '55', '', '4', 'Grande Bretagne', '22', '21', '22', '65', '', '5', 'Russie', '20', '28', '23', '71', '', '6', 'Australie', '17', '7', '21', '45', '', '7', 'Pays-Bas', '10', '12', '14', '36', '', '8', 'France', '10', '12', '11', '33', '', '9', 'Italie', '10', '10', '20', '40', '', '10', 'Allemagne', '10', '10', '16', '36', '', '11', 'Canada', '7', '6', '11', '24', '', '12', 'Brésil', '7', '6', '8', '21', '', '13', 'Nouvelle-Zélande', '7', '6', '7', '20', '', '14', 'Cuba', '7', '3', '5', '15', '', '15', 'Hongrie', '6', '7', '7', '20', '', '16', 'Corée du Sud', '6', '4', '10', '20', '', '17', 'Pologne', '4', '5', '5', '14', '', '18', 'République Tchèque', '4', '5', '3', '12', '', '19', 'Kenya', '4', '4', '2', '10', '', '20', 'Norvège', '4', '2', '2', '8', '', '21', 'Jamaïque', '4', '1', '4', '9', '', '22', 'Espagne', '3', '8', '6', '17', '', '23', 'Suède', '3', '6', '

##### Création du dataframe

In [9]:
Place = [td for td in ele[0::7]]
Pays = [td for td in ele[1::7]]
Or = [td for td in ele[2::7]]
Argent = [td for td in ele[3::7]]
Bronze = [td for td in ele[4::7]]
Total = [td for td in ele[5::7]]
df = pd.DataFrame(dict(zip(col, [Place, Pays, Or, Argent, Bronze, Total])))
df

,Place,Pays,Or,Argent,Bronze,Total
0,1,Etats Unis,38,41,34,113
1,2,Chine,36,32,18,86
2,3,Japon,27,12,16,55
3,4,Grande Bretagne,22,21,22,65
4,5,Russie,20,28,23,71
...,...,...,...,...,...,...
88,86,Côte d'Ivoire,0,0,1,1
89,86,Ghana,0,0,1,1
90,86,Grenade,0,0,1,1
91,86,Moldavie,0,0,1,1


### Récupération de tous les liens des différents jeux olympiques d'été

In [10]:
# on veut récupérer le ul qui se trouve après <label for="tab1dc" class="close" accesskey="3"><img src="images/trans.gif" alt=""></label>
label = soup.find('label', {'for': 'tab1dc', 'class': 'close', 'accesskey': '3'})
ul = label.find_next_sibling('ul')
# on récupère le href et le title de chaque lien
links = [(a['href'], a['title']) for a in ul.find_all('a')]
links

[('2021-tokyo-jeux-olympiques-d-ete-s16-c0-b0-j0-u323.html', '2021 - Tokyo'),
 ('2016-rio-de-janeiro-jeux-olympiques-d-ete-s16-c0-b0-j0-u289.html',
  '2016 - Rio de Janeiro'),
 ('2012-londres-jeux-olympiques-d-ete-s16-c0-b0-j0-u281.html',
  '2012 - Londres'),
 ('2008-pekin-jeux-olympiques-d-ete-s16-c0-b0-j0-u218.html', '2008 - Pékin'),
 ('2004-athenes-jeux-olympiques-d-ete-s16-c0-b0-j0-u10.html',
  '2004 - Athènes'),
 ('2000-sydney-jeux-olympiques-d-ete-s16-c0-b0-j0-u12.html', '2000 - Sydney'),
 ('1996-atlanta-jeux-olympiques-d-ete-s16-c0-b0-j0-u22.html',
  '1996 - Atlanta'),
 ('1992-barcelone-jeux-olympiques-d-ete-s16-c0-b0-j0-u30.html',
  '1992 - Barcelone'),
 ('1988-seoul-jeux-olympiques-d-ete-s16-c0-b0-j0-u38.html', '1988 - Séoul'),
 ('1984-los-angeles-jeux-olympiques-d-ete-s16-c0-b0-j0-u46.html',
  '1984 - Los Angeles'),
 ('1980-moscou-jeux-olympiques-d-ete-s16-c0-b0-j0-u55.html', '1980 - Moscou'),
 ('1976-montreal-jeux-olympiques-d-ete-s16-c0-b0-j0-u63.html',
  '1976 - Montréal')

In [11]:
len(links)

30

### Extraction des données de chaque page dans le dataframe

In [12]:
# # on boucle sur chaque lien pour récupérer les données
# def get_data_from_link(link, data=[], min=0, max=1):
#     for link in links[min:max] :
#         #print(link[0])
#         url = 'https://www.les-sports.info/' + link[0]
#         page = requests.get(url)
#         soup = bs(page.content, 'html.parser')
#         tables = soup.find_all('table', {'class': 'table-style-2'})
#         table = tables[1]
#         col = [td.get_text().strip() for td in table.select("th")]
#         ele = [td.get_text() for td in table.select("td")]
#         Place = [td for td in ele[0::7]]
#         Pays = [td for td in ele[1::7]]
#         Or = [td for td in ele[2::7]]
#         Argent = [td for td in ele[3::7]]
#         Bronze = [td for td in ele[4::7]]
#         Total = [td for td in ele[5::7]]
#         df = pd.DataFrame(dict(zip(col, [Place, Pays, Or, Argent, Bronze, Total])))
#         # on crée une colonne lieu et une colonne année : le titre du lien est sous la forme 1972 - Munich
#         df['Lieu'] = link[1].split(' - ')[1]
#         print(df['Lieu'])
#         df['Annee'] = link[1].split(' - ')[0]
#         data.append(df)
#         # on affiche une barre de progression
#         print(f'{len(data)}/{len(links)}')
#     return data

In [13]:
# # on appelle la fonction
# data = []
# data = get_data_from_link(links, data, min=0, max=len(links))

In [14]:
# on boucle sur chaque lien pour récupérer les données
def get_data_from_link(links, data=[], min=0, max=1):
    for i in tqdm(range(min,max), desc = 'Scraping en cours de progression'):
        url = 'https://www.les-sports.info/' + links[i][0]
        page = requests.get(url)
        soup = bs(page.content, 'html.parser')
        tables = soup.find_all('table', {'class': 'table-style-2'})
        table = tables[1]
        col = [td.get_text().strip() for td in table.select("th")]
        ele = [td.get_text() for td in table.select("td")]
        Place = [td for td in ele[0::7]]
        Pays = [td for td in ele[1::7]]
        Or = [td for td in ele[2::7]]
        Argent = [td for td in ele[3::7]]
        Bronze = [td for td in ele[4::7]]
        Total = [td for td in ele[5::7]]
        df = pd.DataFrame(dict(zip(col, [Place, Pays, Or, Argent, Bronze, Total])))
        # on crée une colonne lieu et une colonne année : le titre du lien est sous la forme 1972 - Munich
        df['Lieu'] = links[i][1].split(' - ')[1]
        #print(df['Lieu'])
        df['Annee'] = links[i][1].split(' - ')[0]
        data.append(df)
        # on affiche une barre de progression
        #print(f'{len(data)}/{len(links)}')
    return data

In [15]:
# on appelle la fonction
data = []
data = get_data_from_link(links, data, min=0, max=len(links))

Scraping en cours de progression:   0%|          | 0/30 [00:00<?, ?it/s]

Scraping en cours de progression: 100%|██████████| 30/30 [00:51<00:00,  1.70s/it]


In [16]:
# On transforme data en dataframe
df = pd.concat(data)
df

,Place,Pays,Or,Argent,Bronze,Total,Lieu,Annee
0,1,Etats Unis,38,41,34,113,Tokyo,2021
1,2,Chine,36,32,18,86,Tokyo,2021
2,3,Japon,27,12,16,55,Tokyo,2021
3,4,Grande Bretagne,22,21,22,65,Tokyo,2021
4,5,Russie,20,28,23,71,Tokyo,2021
...,...,...,...,...,...,...,...,...
5,6,Hongrie,2,1,3,6,Athènes,1896
6,7,Autriche,2,1,2,5,Athènes,1896
7,8,Australie,2,0,1,3,Athènes,1896
8,9,Danemark,1,2,3,6,Athènes,1896


### Exploration de la donnée

##### On affiche tous les scores des médailles obtenues par la France aux JO

In [17]:
df[df['Pays'] == 'France']

,Place,Pays,Or,Argent,Bronze,Total,Lieu,Annee
7,8,France,10,12,11,33,Tokyo,2021
6,7,France,10,18,14,42,Rio de Janeiro,2016
6,7,France,11,11,13,35,Londres,2012
9,10,France,7,18,19,44,Pékin,2008
6,7,France,11,9,14,34,Athènes,2004
5,6,France,13,14,11,38,Sydney,2000
4,5,France,15,8,15,38,Atlanta,1996
11,12,France,8,5,16,29,Barcelone,1992
12,12,France,8,5,16,29,Barcelone,1992
8,9,France,6,4,6,16,Séoul,1988


In [18]:
len(df[df['Pays'] == 'France'])

31

In [19]:
print('Il y a eu en tout {} jeux olympiques'.format(df['Annee'].nunique()))

Il y a eu en tout 30 jeux olympiques


Etrange car la france a participé à tous les JO d'été depuis 1896. En regardant le tableau, on s'apercoit que les lignes des jeux de Barcelone sont dédoublées

Après vérification, il n'y a qu'un lien html ave Barcelone. On drop les duplicates

In [20]:
# on drop les duplicates de df
df = df.drop_duplicates()

In [24]:
# on revérifie avec le nombre de participations de la France
len(df[df['Pays'] == 'France'])

30

In [25]:
print('En moyenne, la France a été classée', round(df[df['Pays'] == 'France']['Place'].astype(int).mean(), 2), 'ème sur les 30 derniers JO')

En moyenne, la France a été classée 8.2 ème sur les 30 derniers JO
